# Building MOP-Terphenyl Polymers using mbuild

Here we use `mbuild` to read in a SMILES string of a terphenyl monomer and build an n-residue polymer from the monomer.

In [15]:
import mbuild as mb
from mbuild.lib.recipes.polymer import Polymer

First we get the MOP-terphenyl monomer from a smiles string.

In [16]:
comp = mb.load('C(=O)c1ccc(c2c(cc(OC)cc2)c2cccc(c2)[C@@H](N)C)cc1', smiles = True, name="MOP")

I output all the indexes of hydrogen atoms because we will uses these indexes for extending the polymer later on.

In [17]:
for i, atom in enumerate(comp):
    if atom.name == "H":
        print(i, atom)

25 <H pos=([-0.2453 -0.5566  0.0782]), 1 bonds, id: 139929034700208>
26 <H pos=([-0.1188 -0.3599  0.1631]), 1 bonds, id: 139929034699584>
27 <H pos=([-0.0695 -0.1196  0.1639]), 1 bonds, id: 139929034699920>
28 <H pos=([ 0.0728  0.3003 -0.0153]), 1 bonds, id: 139926874523632>
29 <H pos=([-0.1463  0.6895  0.0515]), 1 bonds, id: 139926874522864>
30 <H pos=([-0.2465  0.5765  0.1501]), 1 bonds, id: 139929034573616>
31 <H pos=([-0.2739  0.5909 -0.0294]), 1 bonds, id: 139929034573856>
32 <H pos=([-0.3456  0.3696  0.0561]), 1 bonds, id: 139929034575440>
33 <H pos=([-0.3889  0.13    0.0462]), 1 bonds, id: 139929034575104>
34 <H pos=([-0.0317 -0.0545 -0.2148]), 1 bonds, id: 139929034574432>
35 <H pos=([ 0.1565 -0.2054 -0.2646]), 1 bonds, id: 139929034574096>
36 <H pos=([ 0.3579 -0.205  -0.1237]), 1 bonds, id: 139929034574048>
37 <H pos=([0.1828 0.1017 0.1242]), 1 bonds, id: 139929034573424>
38 <H pos=([0.3972 0.0289 0.1781]), 1 bonds, id: 139929034573184>
39 <H pos=([ 0.4395 -0.2537  0.0963]), 1

`mbuild` comes with a nice tool to visualize Compounds built into jupyter-notebooks

In [18]:
comp.visualize(show_ports=True)

/home/lenny/anaconda3/envs/terphenyl/lib/python3.9/site-packages/mbuild/compound.py:400: UserWarning: Some particle of <C pos=([-0.3002 -0.4911  0.0117]), 3 bonds, id: 139928956382464> does not have a charge.They will not be accounted for during this calculation.
  warn(
/home/lenny/anaconda3/envs/terphenyl/lib/python3.9/site-packages/mbuild/compound.py:400: UserWarning: Some particle of <O pos=([-0.3879 -0.5407 -0.0639]), 1 bonds, id: 139929034723488> does not have a charge.They will not be accounted for during this calculation.
  warn(
/home/lenny/anaconda3/envs/terphenyl/lib/python3.9/site-packages/mbuild/compound.py:400: UserWarning: Some particle of <C pos=([-0.2717 -0.3465  0.0106]), 3 bonds, id: 139929034723632> does not have a charge.They will not be accounted for during this calculation.
  warn(
/home/lenny/anaconda3/envs/terphenyl/lib/python3.9/site-packages/mbuild/compound.py:400: UserWarning: Some particle of <C pos=([-0.1735 -0.2945  0.0961]), 3 bonds, id: 139929034723776>

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

We also make the two capping compounds using SMILES strings:

In [5]:
cap_o = mb.load('CO', smiles = True)
cap_o.visualize()

/home/lenny/anaconda3/envs/terphenyl/lib/python3.9/site-packages/mbuild/compound.py:400: UserWarning: Some particle of <C pos=([-0.0357 -0.0032 -0.0017]), 4 bonds, id: 139926874409328> does not have a charge.They will not be accounted for during this calculation.
  warn(
/home/lenny/anaconda3/envs/terphenyl/lib/python3.9/site-packages/mbuild/compound.py:400: UserWarning: Some particle of <O pos=([ 0.0958 -0.0437 -0.0266]), 2 bonds, id: 139926874409424> does not have a charge.They will not be accounted for during this calculation.
  warn(
/home/lenny/anaconda3/envs/terphenyl/lib/python3.9/site-packages/mbuild/compound.py:400: UserWarning: Some particle of <H pos=([-0.0539  0.0029  0.1078]), 1 bonds, id: 139926874409568> does not have a charge.They will not be accounted for during this calculation.
  warn(
/home/lenny/anaconda3/envs/terphenyl/lib/python3.9/site-packages/mbuild/compound.py:400: UserWarning: Some particle of <H pos=([-0.1055 -0.0774 -0.0454]), 1 bonds, id: 139926874409712>

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [6]:
cap_n = mb.load('CC(C)(C)OC=O', smiles = True)
cap_n.visualize()

/home/lenny/anaconda3/envs/terphenyl/lib/python3.9/site-packages/mbuild/compound.py:400: UserWarning: Some particle of <C pos=([-0.077   0.152  -0.0536]), 4 bonds, id: 139926874543440> does not have a charge.They will not be accounted for during this calculation.
  warn(
/home/lenny/anaconda3/envs/terphenyl/lib/python3.9/site-packages/mbuild/compound.py:400: UserWarning: Some particle of <C pos=([-0.0119  0.0207 -0.0076]), 4 bonds, id: 139926874543536> does not have a charge.They will not be accounted for during this calculation.
  warn(
/home/lenny/anaconda3/envs/terphenyl/lib/python3.9/site-packages/mbuild/compound.py:400: UserWarning: Some particle of <C pos=([-0.0768 -0.0231  0.1251]), 4 bonds, id: 139926874543680> does not have a charge.They will not be accounted for during this calculation.
  warn(
/home/lenny/anaconda3/envs/terphenyl/lib/python3.9/site-packages/mbuild/compound.py:400: UserWarning: Some particle of <C pos=([-0.0333 -0.0863 -0.1164]), 4 bonds, id: 139926874543824>

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

Here we use the `Polymer` object to build a hexamer from the molecules we built in the previous cells. `Polymer.add_monomer()` is used to add the monomers to the polymer object. `Polymer.add_end_groups()` adds the capping groups to the object with `"head"` and `"tail"` labels for the each end of the polymer. `replace = True` will replace the specified atoms with the next residue. `indices` is used to specify which atom will be replaced in each group. Finally, when we call `Polymer.build()`, the polymer is built with the specified `n` residues and the provided capping residues. `sequence` is used if multiple monomers are provided.

In [20]:
chain = Polymer()
chain.add_monomer(compound=comp,
                  indices=[25, 39],
                  separation=.15,
                  replace=True,
                  orientation = [[1,-0.5,0.6],[1,-0.5,-0.3]])
chain.add_end_groups(compound = cap_o,
                     index = -1,
                     separation=0.15,
                     label="head",
                     duplicate = False
                    )

chain.add_end_groups(compound = cap_n,
                     index = -1,
                     separation=0.15,
                     label="tail",
                     duplicate = False
                    )

chain.build(n=6, sequence='A')

In [21]:
chain.visualize(show_ports=True)

/home/lenny/anaconda3/envs/terphenyl/lib/python3.9/site-packages/mbuild/compound.py:400: UserWarning: Some particle of <C pos=([-0.3002 -0.4911  0.0117]), 3 bonds, id: 139926874380176> does not have a charge.They will not be accounted for during this calculation.
  warn(
/home/lenny/anaconda3/envs/terphenyl/lib/python3.9/site-packages/mbuild/compound.py:400: UserWarning: Some particle of <O pos=([-0.3879 -0.5407 -0.0639]), 1 bonds, id: 139926874378784> does not have a charge.They will not be accounted for during this calculation.
  warn(
/home/lenny/anaconda3/envs/terphenyl/lib/python3.9/site-packages/mbuild/compound.py:400: UserWarning: Some particle of <C pos=([-0.2717 -0.3465  0.0106]), 3 bonds, id: 139926874379648> does not have a charge.They will not be accounted for during this calculation.
  warn(
/home/lenny/anaconda3/envs/terphenyl/lib/python3.9/site-packages/mbuild/compound.py:400: UserWarning: Some particle of <C pos=([-0.1735 -0.2945  0.0961]), 3 bonds, id: 139926874379120>

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

Next we add specific residue labels for the componenets of the `Polymer` object. Here we label the monomers as HEX and the capping moieties as CAP.

In [22]:
print(chain.labels)
for label in chain.labels["monomer"]:
    label.name = "HEX"
    print(label)
for label in chain.labels["Compound"]:
    label.name = "CAP"
    print(label)

OrderedDict([('monomer', [<Compound 44 particles, 46 bonds, non-periodic, id: 139929031064832>, <Compound 44 particles, 46 bonds, non-periodic, id: 139929031064880>, <Compound 44 particles, 46 bonds, non-periodic, id: 139929030664304>, <Compound 44 particles, 46 bonds, non-periodic, id: 139929030489280>, <Compound 44 particles, 46 bonds, non-periodic, id: 139929029470432>, <Compound 44 particles, 46 bonds, non-periodic, id: 139928960662544>]), ('monomer[0]', <Compound 44 particles, 46 bonds, non-periodic, id: 139929031064832>), ('monomer[1]', <Compound 44 particles, 46 bonds, non-periodic, id: 139929031064880>), ('monomer[2]', <Compound 44 particles, 46 bonds, non-periodic, id: 139929030664304>), ('monomer[3]', <Compound 44 particles, 46 bonds, non-periodic, id: 139929030489280>), ('monomer[4]', <Compound 44 particles, 46 bonds, non-periodic, id: 139929029470432>), ('monomer[5]', <Compound 44 particles, 46 bonds, non-periodic, id: 139928960662544>), ('Compound', [<Compound 5 particles,

We save these as a pdb file and provide the names of the residues to include in the file.

In [29]:
chain.save("mop_hexamer_mbuild.pdb", overwrite=True, residues=["HEX", "CAP"])

Using Gromacs I generate a new gro file with the correct residue labels. `mbuild` doesn't seem to label residues correctly when writing `.gro` files.

In [30]:
! gmx editconf -f mop_hexamer_mbuild.pdb -o mop_hexamer_mbuild.gro

                      :-) GROMACS - gmx editconf, 2022 (-:

Executable:   /usr/local/gromacs/bin/gmx
Data prefix:  /usr/local/gromacs
Working dir:  /home/lenny/Research/heteropolymer_simulations/examples/build_polymer
Command line:
  gmx editconf -f mop_hexamer_mbuild.pdb -o mop_hexamer_mbuild.gro

Note that major changes are planned in future for editconf, to improve usability and utility.
Read 285 atoms
Volume: 14.8707 nm^3, corresponds to roughly 6600 electrons
No velocities found

Back Off! I just backed up mop_hexamer_mbuild.gro to ./#mop_hexamer_mbuild.gro.4#

GROMACS reminds you: "We Look Pretty Sharp In These Clothes" (F. Zappa)



Lastly, I use openBabel to generate a `.mol` file for use in the OpenFF parameter assignment workflow.

In [31]:
! obabel -ipdb mop_hexamer_mbuild.pdb -omol mop_hexamer_mbuild.mol -O mop_hexamer_mbuild.mol

*** Open Babel Error  in ReadMolecule
  ERROR: not a valid PDB file

1 molecule converted
